In [1]:
import scanpy as sc
import pandas as pd
# from scipy.io import mmread
import numpy as np
import scipy.sparse as sp
import os

In [2]:
adata = sc.read_h5ad("./05a-aml-BM-annotated-complete.h5ad")
# adata = sc.read_h5ad("./content/input_data/03a-aml-BM-integrated-complete.h5ad")
adata
# adata.obs = adata.obs.rename(columns={"patient": "individual"})

/home/nghlm/yard/anaconda3/envs/base_gao/lib/python3.12/site-packages/anndata/compat/__init__.py:371: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  warn(


AnnData object with n_obs × n_vars = 49641 × 2000
    obs: 'prject', 'nUMI', 'nGene', 'cellID', 'Doublet_Score', 'Doublet_Class', 'log10GenesPerUMI', 'mitoRatio', 'riboRatio1', 'riboRatio2', 'riboRatio3', 'riboRatio4', 'riboRatio', 'RNA_snn_res.0', 'RNA_snn_res.0.1', 'RNA_snn_res.0.2', 'RNA_snn_res.0.3', 'RNA_snn_res.0.4', 'RNA_snn_res.0.5', 'RNA_snn_res.0.6', 'RNA_snn_res.0.7', 'RNA_snn_res.0.8', 'RNA_snn_res.0.9', 'RNA_snn_res.1', 'seurat_clusters', 'class1', 'sample', 'source', 'S.Score', 'G2M.Score', 'Phase', 'RNA_snn_res.1.2', 'RNA_snn_res.1.4', 'RNA_snn_res.1.6', 'RNA_snn_res.1.8', 'RNA_snn_res.2', 'cellType', 'dcellType', 'cell', 'patient', 'timing', 'category', 'response', 'sub.response', 'date', 'start', 'daysSinceStart', 'blast', 'age', 'sex', 'FAB', 'cellCount', 'monthsSinceStart', 'relapse', 'daysStartRelapse', 'monthsStartRelapse', 'azimuthNames', 'mut', 'p53.mut'
    var: 'vst.mean', 'vst.variance', 'vst.variance.expected', 'vst.variance.standardized', 'vst.variable'
    

In [3]:
print(adata.raw is not None)
if adata.raw is not None:
    print(adata.raw.shape)   # 期望 ~ (49641, 27390)

old = adata  # 备份
assert old.raw is not None

# 1) 用 raw 构建一个“全基因”的新对象
adata_full = old.raw.to_adata()   # 形状会是 (49641, 27390)

# 2) 把 obs/obsm/uns 等元信息带回来（这些与基因数无关）
adata_full.obs = old.obs.copy()
adata_full.obsm = old.obsm.copy()    # 保留 X_harmony / X_pca
adata_full.uns = old.uns.copy()

# （可选）3) 把原 2k HVG 的 log1p 主矩阵保存为一个对齐到全基因的 layer，方便日后对照
hvg_genes = old.var_names
inter, idx_full, idx_hvg = np.intersect1d(adata_full.var_names, hvg_genes, return_indices=True)

# 建一个全 0 的稀疏层，然后把 2k HVG 的矩阵放到对应列
if sp.issparse(old.X):
    layer = sp.csr_matrix((adata_full.n_obs, adata_full.n_vars), dtype=old.X.dtype)
    layer[:, idx_full] = old.X[:, idx_hvg]
else:
    layer = np.zeros((adata_full.n_obs, adata_full.n_vars), dtype=old.X.dtype)
    layer[:, idx_full] = old.X[:, idx_hvg]

adata_full.layers["hvg_log1p"] = layer

# 4) 用新的全基因对象替换
adata = adata_full

# 检查
adata  # 现在应为 (49641, 27390)
list(adata.layers.keys())  # 包含 'hvg_log1p'（可选）
list(adata.obsm.keys())    # 仍有 'X_harmony'、'X_pca'



True
(49641, 27390)


['X_harmony', 'X_pca', 'X_tsne', 'X_umap']

In [4]:
adata

AnnData object with n_obs × n_vars = 49641 × 27390
    obs: 'prject', 'nUMI', 'nGene', 'cellID', 'Doublet_Score', 'Doublet_Class', 'log10GenesPerUMI', 'mitoRatio', 'riboRatio1', 'riboRatio2', 'riboRatio3', 'riboRatio4', 'riboRatio', 'RNA_snn_res.0', 'RNA_snn_res.0.1', 'RNA_snn_res.0.2', 'RNA_snn_res.0.3', 'RNA_snn_res.0.4', 'RNA_snn_res.0.5', 'RNA_snn_res.0.6', 'RNA_snn_res.0.7', 'RNA_snn_res.0.8', 'RNA_snn_res.0.9', 'RNA_snn_res.1', 'seurat_clusters', 'class1', 'sample', 'source', 'S.Score', 'G2M.Score', 'Phase', 'RNA_snn_res.1.2', 'RNA_snn_res.1.4', 'RNA_snn_res.1.6', 'RNA_snn_res.1.8', 'RNA_snn_res.2', 'cellType', 'dcellType', 'cell', 'patient', 'timing', 'category', 'response', 'sub.response', 'date', 'start', 'daysSinceStart', 'blast', 'age', 'sex', 'FAB', 'cellCount', 'monthsSinceStart', 'relapse', 'daysStartRelapse', 'monthsStartRelapse', 'azimuthNames', 'mut', 'p53.mut'
    var: 'vst.mean', 'vst.variance', 'vst.variance.expected', 'vst.variance.standardized', 'vst.variable'
   

In [5]:
adata.obs['sample'].value_counts()

sample
HCBM1 control    9828
Pt 15 pre        9091
HCBM2 control    8549
HCBM3 control    4304
Pt 13 post       2482
Pt 17 pre2       1801
Pt 7 pre         1735
Pt 14 post       1487
Pt 19 post       1441
Pt 21 post       1328
Pt 11 pre        1305
Pt 12 pre        1120
Pt 12 post       1018
Pt 20 post        883
Pt 9 pre          781
Pt 11 post        583
Pt 5 pre          553
Pt 17 post        330
Pt 6 post         324
Pt 13 pre         241
Pt 6 pre          204
Pt 4 pre          161
Pt 1 post          92
Name: count, dtype: int64

In [7]:
patient2ven = {
    # ==== nonresponder ====
    "Pt 21": "nonresponder",
    "Pt 20": "nonresponder",
    "Pt 19": "nonresponder",
    "Pt 17": "nonresponder",
    "Pt 15": "nonresponder",
    "Pt 10": "nonresponder",
    "Pt 7":  "nonresponder",
    "Pt 18": "nonresponder",
    "Pt 16": "nonresponder",
    "Pt 22": "nonresponder",

    # ==== responder ====
    "Pt 6":  "responder",
    "Pt 5":  "responder",
    "Pt 4":  "responder",
    "Pt 1":  "responder",
    "Pt 9":  "responder",
    "Pt 12": "responder",
    "Pt 13": "responder",
    "Pt 14": "responder",
    "Pt 11": "responder",
    "Pt 2":  "responder",
    "Pt 3":  "responder",
    "Pt 8":  "responder",
}


In [8]:
adata.obs["venetoclax"] = adata.obs["patient"].map(patient2ven)


In [9]:
adata.obs.loc[adata.obs["venetoclax"]  == "nonresponder", "patient"].value_counts()


patient
Pt 15    9091
Pt 17    2131
Pt 7     1735
Pt 19    1441
Pt 21    1328
Pt 20     883
Name: count, dtype: int64

In [10]:
adata.obs.loc[adata.obs["venetoclax"]  == "responder", 'sample'].value_counts()


sample
Pt 13 post    2482
Pt 14 post    1487
Pt 11 pre     1305
Pt 12 pre     1120
Pt 12 post    1018
Pt 9 pre       781
Pt 11 post     583
Pt 5 pre       553
Pt 6 post      324
Pt 13 pre      241
Pt 6 pre       204
Pt 4 pre       161
Pt 1 post       92
Name: count, dtype: int64

In [11]:
patient2treat = {
    "Pt 15 pre":  "pre",
    "Pt 17 pre2": "pre",   # pre2 也算 pre
    "Pt 7 pre":   "pre",
    "Pt 19 post": "post",
    "Pt 21 post": "post",
    "Pt 20 post": "post",
    "Pt 17 post": "post",
    "Pt 13 post": "post",
    "Pt 14 post": "post",
    "Pt 11 pre":  "pre",
    "Pt 12 pre":  "pre",
    "Pt 12 post": "post",
    "Pt 9 pre":   "pre",
    "Pt 11 post": "post",
    "Pt 5 pre":   "pre",
    "Pt 6 post":  "post",
    "Pt 13 pre":  "pre",
    "Pt 6 pre":   "pre",
    "Pt 4 pre":   "pre",
    "Pt 1 post":  "post",
}


In [12]:
adata.obs["treat"] = adata.obs['sample'].map(patient2treat)


In [13]:
adata.obs["treat"]

healthy4_control_NLBM4_AAACCTGAGAGGGCTT-1     NaN
healthy4_control_NLBM4_AAACCTGAGAGTCGGT-1     NaN
healthy4_control_NLBM4_AAACCTGAGCGTTTAC-1     NaN
healthy4_control_NLBM4_AAACCTGAGTGAACAT-1     NaN
healthy4_control_NLBM4_AAACCTGCAAACCTAC-1     NaN
                                             ... 
Patient8_post_TTTGGAGGTAGCACAG-1             post
Patient8_post_TTTGGTTAGCAGTACG-1             post
Patient8_post_TTTGGTTCACCAGACC-1             post
Patient8_post_TTTGGTTCATCCTATT-1             post
Patient8_post_TTTGGTTTCACTCACC-1             post
Name: treat, Length: 49641, dtype: object

In [14]:
adata

AnnData object with n_obs × n_vars = 49641 × 27390
    obs: 'prject', 'nUMI', 'nGene', 'cellID', 'Doublet_Score', 'Doublet_Class', 'log10GenesPerUMI', 'mitoRatio', 'riboRatio1', 'riboRatio2', 'riboRatio3', 'riboRatio4', 'riboRatio', 'RNA_snn_res.0', 'RNA_snn_res.0.1', 'RNA_snn_res.0.2', 'RNA_snn_res.0.3', 'RNA_snn_res.0.4', 'RNA_snn_res.0.5', 'RNA_snn_res.0.6', 'RNA_snn_res.0.7', 'RNA_snn_res.0.8', 'RNA_snn_res.0.9', 'RNA_snn_res.1', 'seurat_clusters', 'class1', 'sample', 'source', 'S.Score', 'G2M.Score', 'Phase', 'RNA_snn_res.1.2', 'RNA_snn_res.1.4', 'RNA_snn_res.1.6', 'RNA_snn_res.1.8', 'RNA_snn_res.2', 'cellType', 'dcellType', 'cell', 'patient', 'timing', 'category', 'response', 'sub.response', 'date', 'start', 'daysSinceStart', 'blast', 'age', 'sex', 'FAB', 'cellCount', 'monthsSinceStart', 'relapse', 'daysStartRelapse', 'monthsStartRelapse', 'azimuthNames', 'mut', 'p53.mut', 'venetoclax', 'treat'
    var: 'vst.mean', 'vst.variance', 'vst.variance.expected', 'vst.variance.standardiz

In [42]:
adata.obs['dcellType'].value_counts()

dcellType
CD4               14678
Late Erythroid     6026
NK                 4155
CD8 CTL            4113
CD14 Mono          3773
B cells            3219
BC2                2364
CD8 Mem            1710
CD8 Naive          1570
CD8 Ex             1281
BC3                1103
DC                  905
BC1                 884
BC4                 883
GMP                 816
CD8 EM              709
pre B               492
CD16 Mono           292
Macrophage          253
Prog Mk             247
Plasma              168
Name: count, dtype: int64

In [43]:
# 先筛选 treat=post 且 venetoclax=nonresponder
subset = adata.obs[
    (adata.obs["treat"] == "post") &
    (adata.obs["venetoclax"] == "nonresponder")
]

# 再筛选 CD8 类的细胞
cd8_subset = subset[subset["dcellType"].str.startswith("CD8")]

# 统计个数
cd8_count = cd8_subset.shape[0]
print("CD8 类细胞数量:", cd8_count)


CD8 类细胞数量: 252


In [44]:
cd8_counts = cd8_subset["dcellType"].value_counts()
print(cd8_counts)


dcellType
CD8 CTL      81
CD8 Naive    71
CD8 EM       43
CD8 Mem      31
CD8 Ex       26
Name: count, dtype: int64


In [ ]:
import numpy as np
import pandas as pd

def compute_cd8_fraction(adata, treat, venetoclax):
    """
    计算指定 treat & venetoclax 条件下，
    各 sample 内 CD8 类占比，并返回均值和离散程度
    """
    # 1) 条件筛选
    subset = adata.obs[
        (adata.obs["treat"] == treat) &
        (adata.obs["venetoclax"] == venetoclax)
    ]
    
    # 2) 按 sample 计算 CD8 占比
    def cd8_fraction(df):
        total = len(df)
        cd8 = (df["dcellType"].str.startswith("CD8")).sum()
        return cd8 / total if total > 0 else np.nan

    fractions = subset.groupby("sample").apply(cd8_fraction)

    # 3) 统计
    return {
        "treat": treat,
        "venetoclax": venetoclax,
        "mean": fractions.mean(),
        "std": fractions.std(),
        "sem": fractions.sem(),
        "n_samples": fractions.shape[0],
        "fractions": fractions
    }


results = []
for cond in [("pre", "responder"), ("post", "responder"), ("pre", "nonresponder"), ("post", "nonresponder")]:
    results.append(compute_cd8_fraction(adata, *cond))

# 汇总成表格（均值 ± 标准差）
summary = pd.DataFrame([
    {
        "treat": r["treat"],
        "venetoclax": r["venetoclax"],
        "n_samples": r["n_samples"],
        "mean": r["mean"],
        "std": r["std"],
        "sem": r["sem"]
    }
    for r in results
])

print(summary)


  treat    venetoclax  n_samples      mean       std       sem
0   pre     responder          7  0.101879  0.054255  0.020506
1  post     responder          6  0.242516  0.056311  0.022989
2   pre  nonresponder          3  0.147779  0.050377  0.029085
3  post  nonresponder          4  0.069725  0.040696  0.020348


/tmp/ipykernel_37254/3211884323.py:21: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  fractions = subset.groupby("sample").apply(cd8_fraction)
/tmp/ipykernel_37254/3211884323.py:21: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  fractions = subset.groupby("sample").apply(cd8_fraction)
/tmp/ipykernel_37254/3211884323.py:21: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behav

In [48]:
import numpy as np
import pandas as pd

def compute_cd8_fraction(adata, treat, venetoclax):
    """
    计算指定 treat & venetoclax 条件下，
    各 sample 内 CD8 类占比，并返回均值和离散程度
    """
    # 1) 条件筛选
    subset = adata.obs[adata.obs["venetoclax"] == venetoclax
    ]
    
    # 2) 按 sample 计算 CD8 占比
    def cd8_fraction(df):
        total = len(df)
        cd8 = (df["dcellType"].str.startswith("CD8")).sum()
        return cd8 / total if total > 0 else np.nan

    fractions = subset.groupby("sample").apply(cd8_fraction)

    # 3) 统计
    return {
        "treat": treat,
        "venetoclax": venetoclax,
        "mean": fractions.mean(),
        "std": fractions.std(),
        "sem": fractions.sem(),
        "n_samples": fractions.shape[0],
        "fractions": fractions
    }


results = []
for cond in [("pre", "responder"), ("post", "responder"), ("pre", "nonresponder"), ("post", "nonresponder")]:
    results.append(compute_cd8_fraction(adata, *cond))

# 汇总成表格（均值 ± 标准差）
summary = pd.DataFrame([
    {
        "treat": r["treat"],
        "venetoclax": r["venetoclax"],
        "n_samples": r["n_samples"],
        "mean": r["mean"],
        "std": r["std"],
        "sem": r["sem"]
    }
    for r in results
])

print(summary)


  treat    venetoclax  n_samples      mean       std       sem
0   pre     responder         13  0.166789  0.090100  0.024989
1  post     responder         13  0.166789  0.090100  0.024989
2   pre  nonresponder          7  0.103177  0.058436  0.022087
3  post  nonresponder          7  0.103177  0.058436  0.022087


/tmp/ipykernel_37254/4266838807.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  fractions = subset.groupby("sample").apply(cd8_fraction)
/tmp/ipykernel_37254/4266838807.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  fractions = subset.groupby("sample").apply(cd8_fraction)
/tmp/ipykernel_37254/4266838807.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behav

In [61]:
import numpy as np

# 1) 取出 treat 为 NaN 的细胞
subset = adata.obs[~adata.obs["treat"].isna()]

# 2) 定义一个“是不是 CD8 类”布尔条件
is_cd8 = subset["dcellType"].str.startswith("CD8")

# 3) 按 sample 分组，计算 CD8 占比
cd8_frac_by_sample = (
    subset.groupby("sample")
    .apply(lambda df: (df["dcellType"].str.startswith("CD8")).sum() / len(df))
)

# 4) 计算平均值和标准差
mean_frac = cd8_frac_by_sample.mean()
std_frac = cd8_frac_by_sample.std()

print("各 sample 的 CD8 占比：")
print(cd8_frac_by_sample)
print(f"\n平均占比: {mean_frac:.4f}, 标准差: {std_frac:.4f}")


各 sample 的 CD8 占比：
sample
Pt 1 post     0.173913
Pt 11 post    0.334477
Pt 11 pre     0.015326
Pt 12 post    0.277996
Pt 12 pre     0.052679
Pt 13 post    0.232877
Pt 13 pre     0.124481
Pt 14 post    0.225958
Pt 15 pre     0.194478
Pt 17 post    0.115152
Pt 17 pre2    0.094392
Pt 19 post    0.054129
Pt 20 post    0.021518
Pt 21 post    0.088102
Pt 4 pre      0.136646
Pt 5 pre      0.081374
Pt 6 post     0.209877
Pt 6 pre      0.132353
Pt 7 pre      0.154467
Pt 9 pre      0.170294
dtype: float64

平均占比: 0.1445, 标准差: 0.0847


/tmp/ipykernel_37254/1334706333.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda df: (df["dcellType"].str.startswith("CD8")).sum() / len(df))


In [62]:
import pandas as pd

results = {}

for condition_name, cond_filter in {
    "pre_responder":  (adata.obs["treat"]=="pre") & (adata.obs["venetoclax"]=="responder"),
    "post_responder": (adata.obs["treat"]=="post") & (adata.obs["venetoclax"]=="responder")
}.items():
    
    # 筛选条件
    subset = adata.obs[cond_filter]
    # 只保留 CD8 类
    subset_cd8 = subset[subset["dcellType"].str.startswith("CD8")]

    # 按 sample 和 dcellType 计数
    counts = subset_cd8.groupby(["sample", "dcellType"]).size().unstack(fill_value=0)

    # 每个 sample 内换算成比例
    fracs = counts.div(counts.sum(axis=1), axis=0)

    # 计算平均和标准差
    mean_frac = fracs.mean(axis=0)
    std_frac = fracs.std(axis=0)

    results[condition_name] = pd.DataFrame({"mean": mean_frac, "std": std_frac})

# 查看结果
print("=== Pre responder CD8 composition ===")
print(results["pre_responder"])
print("\n=== Post responder CD8 composition ===")
print(results["post_responder"])


=== Pre responder CD8 composition ===
               mean       std
dcellType                    
CD8 CTL    0.424185  0.090753
CD8 EM     0.118388  0.068762
CD8 Ex     0.179287  0.114754
CD8 Mem    0.095469  0.043741
CD8 Naive  0.182670  0.089484

=== Post responder CD8 composition ===
               mean       std
dcellType                    
CD8 CTL    0.342663  0.053085
CD8 EM     0.153861  0.028298
CD8 Ex     0.132618  0.079719
CD8 Mem    0.109280  0.063935
CD8 Naive  0.261578  0.099154


In [64]:
import pandas as pd

results = {}

for condition_name, cond_filter in {
    "pre_responder":  (adata.obs["treat"]=="pre") & (adata.obs["venetoclax"]=="nonresponder"),
    "post_responder": (adata.obs["treat"]=="post") & (adata.obs["venetoclax"]=="nonresponder")
}.items():
    
    # 筛选条件
    subset = adata.obs[cond_filter]
    # 只保留 CD8 类
    subset_cd8 = subset[subset["dcellType"].str.startswith("CD8")]

    # 按 sample 和 dcellType 计数
    counts = subset_cd8.groupby(["sample", "dcellType"]).size().unstack(fill_value=0)

    # 每个 sample 内换算成比例
    fracs = counts.div(counts.sum(axis=1), axis=0)

    # 计算平均和标准差
    mean_frac = fracs.mean(axis=0)
    std_frac = fracs.std(axis=0)

    results[condition_name] = pd.DataFrame({"mean": mean_frac, "std": std_frac})

# 查看结果
print("=== Pre nonresponder CD8 composition ===")
print(results["pre_responder"])
print("\n=== Post nonresponder CD8 composition ===")
print(results["post_responder"])


=== Pre nonresponder CD8 composition ===
               mean       std
dcellType                    
CD8 CTL    0.359000  0.038045
CD8 EM     0.135723  0.018656
CD8 Ex     0.159427  0.084635
CD8 Mem    0.135733  0.038071
CD8 Naive  0.210117  0.072832

=== Post nonresponder CD8 composition ===
               mean       std
dcellType                    
CD8 CTL    0.352452  0.064939
CD8 EM     0.144681  0.102290
CD8 Ex     0.078666  0.061878
CD8 Mem    0.094692  0.056018
CD8 Naive  0.329510  0.137663


In [ ]:
import pandas as pd

results = {}

for condition_name, cond_filter in {
    "pre_responder":  (adata.obs["treat"]=="pre") & (adata.obs["venetoclax"]=="nonresponder"),
    "post_responder": (adata.obs["treat"]=="post") & (adata.obs["venetoclax"]=="nonresponder")
}.items():
    
    # 筛选条件
    subset = adata.obs[cond_filter]
    # 只保留 CD8 类
    subset_cd8 = subset[subset["dcellType"].str.startswith("CD8")]

    # 按 sample 和 dcellType 计数
    counts = subset_cd8.groupby(["sample", "dcellType"]).size().unstack(fill_value=0)

    # 每个 sample 内换算成比例
    fracs = counts.div(counts.sum(axis=1), axis=0)

    # 计算平均和标准差
    mean_frac = fracs.mean(axis=0)
    std_frac = fracs.std(axis=0)

    results[condition_name] = pd.DataFrame({"mean": mean_frac, "std": std_frac})

# 查看结果
print("=== Pre nonresponder CD8 composition ===")
print(results["pre_responder"])
print("\n=== Post nonresponder CD8 composition ===")
print(results["post_responder"])


In [68]:
import pandas as pd

def summarize_cd8(obs, condition_name, condition_filter):
    # 筛选条件
    subset = obs[condition_filter]
    subset_cd8 = subset[subset["dcellType"].str.startswith("CD8")]

    if subset_cd8.empty:
        return pd.DataFrame()

    # 按 sample & dcellType 计数
    counts = subset_cd8.groupby(["sample", "dcellType"]).size().unstack(fill_value=0)

    # 转换为比例
    fracs = counts.div(counts.sum(axis=1), axis=0)

    # 计算均值和标准差
    mean = fracs.mean(axis=0)
    std = fracs.std(axis=0)

    # 拼接成 "mean±std"
    formatted = mean.combine(std, lambda m, s: f"{m:.3f}±{s:.3f}")
    return formatted.to_frame(condition_name)

# 三种条件
results = []

results.append(
    summarize_cd8(
        adata.obs, 
        "responder",
        adata.obs["venetoclax"]=="responder"
    )
)

results.append(
    summarize_cd8(
        adata.obs,
        "nonresponder",
        adata.obs["venetoclax"]=="nonresponder"
    )
)

results.append(
    summarize_cd8(
        adata.obs,
        "control",
        adata.obs["treat"].isna()
    )
)

# 合并结果表
final = pd.concat(results, axis=1)

# 输出到屏幕（可直接复制到Excel）
print(final)

# 如果想保存成 Excel 文件
# final.to_excel("CD8_composition.xlsx")


             responder nonresponder      control
dcellType                                       
CD8 CTL    0.387±0.084  0.355±0.051  0.479±0.067
CD8 EM     0.135±0.055  0.141±0.073  0.026±0.009
CD8 Ex     0.158±0.099  0.113±0.079  0.135±0.033
CD8 Mem    0.102±0.052  0.112±0.050  0.224±0.023
CD8 Naive  0.219±0.099  0.278±0.124  0.136±0.019


In [69]:
# 额外的条件： treat 不为 NaN
treated = summarize_cd8(
    adata.obs,
    "treated",
    adata.obs["treat"].notna()
)

# 合并到已有结果表
final2 = pd.concat([final, treated], axis=1)

# 转置方便查看
final2_T = final2.T
print(final2_T)

# 保存
# final2_T.to_excel("CD8_composition_with_treated.xlsx")


dcellType         CD8 CTL       CD8 EM       CD8 Ex      CD8 Mem    CD8 Naive
responder     0.387±0.084  0.135±0.055  0.158±0.099  0.102±0.052  0.219±0.099
nonresponder  0.355±0.051  0.141±0.073  0.113±0.079  0.112±0.050  0.278±0.124
control       0.479±0.067  0.026±0.009  0.135±0.033  0.224±0.023  0.136±0.019
treated       0.376±0.074  0.137±0.060  0.142±0.093  0.105±0.050  0.240±0.109


In [ ]:
import scanpy as sc
import pandas as pd
from scipy.io import mmread
counts = mmread("../yard/20220801-hiv-results/GSE171964/GSE171964_countsmatrix_v2.mtx.gz").T.tocsr()  # 转置，确保行是细胞
metadata = pd.read_csv("../yard/20220801-hiv-results/GSE171964/GSE171964_geo_pheno_v2.csv", index_col=0, low_memory=False)
genes = pd.read_csv("../yard/20220801-hiv-results/GSE171964/GSE171964_feats_v2.tsv", sep=" ", quotechar='"')

adata = sc.AnnData(X=counts, obs=metadata, var=pd.DataFrame(index=genes['x'].values))

In [32]:
adata.write("../yard/20220801-hiv-results/GSE171964/GSE171964.h5ad")